In [1]:
import sys
import warnings
from datetime import datetime
import numpy as np
import glob
import h5py
from IPython.display import display, clear_output
import yaml
import matplotlib
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.art3d as art3d
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
import os
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
# Data directory:
#dir = '/global/cfs/cdirs/dune/www/data/2x2/CRS/commission/May2024/self_trigger_m0_test'
#dir = '/global/cfs/cdirs/dune/www/data/2x2/CRS/commission/April2024/self_trigger_test_apr30'
dir1 = '/global/cfs/cdirs/dune/www/data/2x2/nearline/packet/commission/May2024/self_trigger_m0_test'
dir0 = '/global/cfs/cdirs/dune/www/data/2x2/nearline/packet/commission/April2024/self_trigger_test_apr30'
ext = 'packet*2024_05_02*5'
h5_files0 = glob.glob(f'{dir0}/{ext}')
h5_files1 = glob.glob(f'{dir1}/{ext}')
h5_files = h5_files0 + h5_files1
#print(h5_files)
print("Number of H5 Files0:", len(h5_files0))
print("Number of H5 Files1:", len(h5_files1))
print("Number of H5 Files:", len(h5_files))
#print(h5_files)
time_only_list = []
for file in h5_files:
    file_name = file.split("/")[-1]
    date = file_name.split("_CDT")[0]
    time_only = date.split("2024_05_02_")[-1]
    time_only_list.append(time_only)
    
# Pair each string with its original position
indexed_time_only = list(enumerate(time_only_list))

# Sort the pairs by string
sorted_pairs = sorted(indexed_time_only, key=lambda pair: pair[1])

# Separate the positions and sorted strings
time_positions, sorted_strings = zip(*sorted_pairs)
print(time_positions)
print(sorted_strings)

Number of H5 Files0: 14
Number of H5 Files1: 29
Number of H5 Files: 43
(10, 1, 3, 6, 9, 4, 0, 12, 5, 13, 2, 11, 8, 7, 27, 28, 14, 25, 36, 31, 34, 20, 19, 39, 40, 38, 37, 21, 41, 33, 42, 29, 30, 26, 16, 22, 24, 35, 18, 23, 17, 32, 15)
('11_05_50', '11_09_11', '12_00_12', '12_05_47', '12_10_12', '12_20_20', '12_23_05', '12_26_16', '12_28_16', '12_36_05', '12_36_38', '12_37_09', '12_37_40', '12_38_11', '12_46_05', '12_56_07', '13_06_08', '13_16_09', '13_26_10', '13_36_11', '13_46_12', '13_56_13', '14_06_15', '14_16_16', '14_26_17', '14_36_18', '14_46_19', '14_56_20', '15_06_21', '15_26_41', '15_27_18', '15_32_20', '18_18_23', '18_28_25', '18_38_26', '18_48_27', '18_58_29', '19_08_30', '19_18_31', '19_28_32', '19_38_33', '19_48_34', '19_58_35')


In [3]:
with PdfPages('output_full.pdf') as output:
    # Test what's in files
    for i in range(len(h5_files)):
        
        #if i>1: continue
        file_number = time_positions[i]
        file = h5_files[file_number]
        print("------------------------------------")
        print("File:", file)
        f = h5py.File(file,'r')
        print(list(f['packets'].attrs))
        if i==0: 
            print("PACKETS:", f['packets'].dtype.names)
        #print("File keys:", f.keys())
        #print("CONFIGS:", f['configs'].dtype.names)
        #print("Timestamp config:", f['configs']['timestamp'][:])
        #print("IO Group config:", f['configs']['io_group'][:])
        #print("IO Channel config:", f['configs']['io_channel'][:])
        #print("Chip ID config:", f['configs']['chip_id'][:])
        #print("Registers config:", f['configs']['registers'][:])
        #print('MESSAGES:', f['messages'].dtype.names)
        #print("Timestamp message:", f['messages']['timestamp'][:])
        #print("Message message:", f['messages']['message'][:])
        #print("Index message:", f['messages']['index'][:])
        #print("PACKETS:", f['packets'].dtype.names)
        print("Number of packets:", len(f['packets']))
        print("Max Timestamp:", np.max(np.array(f['packets']['timestamp'])))
        print("------------------------------------")
        weird_dw = np.where(np.array(f['packets']['dataword']) == 128)
        datawords = f['packets']['dataword'][weird_dw]
        timestamps = f['packets']['timestamp'][weird_dw]
        chip_ids = f['packets']['chip_id'][weird_dw]
        packet_type = f['packets']['packet_type'][weird_dw]
        print("Number of Packets with Dataword==128:", len(datawords))
        #print("Timestamps length:", len(timestamps))
        print("Packet types represented:", np.unique(np.array(packet_type)))
        
        output.attach_note("File: "+file+"\nNumber of packets: "+str(len(datawords))+"\nNumber of packets w/ dataword==128: "+str(len(datawords))+"\nTypes of packets w/ dw == 128: "+str(np.unique(np.array(packet_type))))
        #plt.hist2d(timestamps, chip_ids, density=datawords)
        #plt.show()
        
        
        #plt.scatter(timestamps, datawords, color='red', s=1)
        #plt.xlabel('Timestamp')
        #plt.ylabel('Dataword')
        #plt.show()
        
        #plt.scatter(chip_ids, datawords, color='blue', s=1)
        #plt.hist2d(chip_ids, datawords, bins=[np.max(chip_ids), 10])
        #plt.xlabel('Chip ID')
        #plt.ylabel('Dataword')
        #plt.show()
        
        plt.scatter(timestamps, chip_ids, color='green', s=1)
        plt.xlabel('Timestamp')
        plt.ylabel('Chip ID')
        output.savefig()
        plt.close()
    
        
        print("------------------------------------")
        
        #print("Packet types:", np.unique(np.array(packet_type)))
        #plt.hist(packet_type, bins=np.max(np.unique(np.array(packet_type))))
        

------------------------------------
File: /global/cfs/cdirs/dune/www/data/2x2/nearline/packet/commission/April2024/self_trigger_test_apr30/packet-0000003-2024_05_02_11_05_50_CDT.hdf5
[]
PACKETS: ('io_group', 'io_channel', 'chip_id', 'packet_type', 'downstream_marker', 'parity', 'valid_parity', 'channel_id', 'timestamp', 'dataword', 'trigger_type', 'local_fifo', 'shared_fifo', 'register_address', 'register_data', 'direction', 'local_fifo_events', 'shared_fifo_events', 'counter', 'fifo_diagnostics_enabled', 'first_packet', 'receipt_timestamp')
Number of packets: 6519256
Max Timestamp: 2147483647
------------------------------------
Number of Packets with Dataword==128: 61108
Packet types represented: [0]
------------------------------------
------------------------------------
File: /global/cfs/cdirs/dune/www/data/2x2/nearline/packet/commission/April2024/self_trigger_test_apr30/packet-0000003-2024_05_02_11_09_11_CDT.hdf5
[]
Number of packets: 281361397
Max Timestamp: 4294967295
--------

KeyboardInterrupt: 